# RF Spectrum Surveillance Monitor
## Project 1: Crowdsourced Wideband Spectrum Surveillance (ElectroSense PSD)

**Pipeline overview:**
```
PSD data → Noise floor estimation → CA-CFAR detection → Event grouping → Feature extraction → Outputs
```

**Dataset:** ElectroSense PSD Spectrum Dataset  
Download: https://zenodo.org/records/7521246

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

import numpy as np
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

from spectrum_monitor import (
    generate_demo_psd, load_psd_data,
    estimate_noise_floor, apply_cfar_full,
    group_events, classify_events, compute_occupancy
)
from visualize import plot_waterfall, plot_noise_floor, plot_event_features
from outputs import save_events_csv, save_sensor_summary

print('Imports OK')

## Step 1 – Load Data

Set `USE_DEMO = True` to use synthetic data.  
Set `USE_DEMO = False` and point `DATA_PATH` at your downloaded `.npz` file.

In [ ]:
USE_DEMO   = True      # ← change to False when you have real data
DATA_PATH  = '../data/psd_data.npz'   # ← path to your ElectroSense file
SENSOR_ID  = 'sensor_001'

if USE_DEMO:
    psd, frequencies, timestamps = generate_demo_psd(n_time=720, n_freq=512)
else:
    psd, frequencies, timestamps = load_psd_data(DATA_PATH)

print(f'PSD shape     : {psd.shape}')
print(f'Freq range    : {frequencies[0]/1e6:.1f} – {frequencies[-1]/1e6:.1f} MHz')
print(f'Time frames   : {len(timestamps)}')
print(f'PSD value range: {psd.min():.1f} to {psd.max():.1f} dB')

## Step 2 – Noise Floor Estimation

**MAD method:** `μ(f,t) = median over sliding window`, robust to intermittent transmissions.  
Increase `window` for more smoothing, or switch to `quantile` for explicit false-alarm control.

In [ ]:
NOISE_WINDOW = 60     # frames
NOISE_METHOD = 'mad'  # 'mad' or 'quantile'

mu, sigma = estimate_noise_floor(psd, window=NOISE_WINDOW, method=NOISE_METHOD)

fig = plot_noise_floor(psd, mu, frequencies)
fig.show()

## Step 3 – CA-CFAR Detection

- `pfa=1e-3` targets 1 false alarm per 1000 cells per frame
- `min_persist=2` removes isolated single-frame spikes

In [ ]:
PFA         = 1e-3   # probability of false alarm
GUARD       = 4      # guard cells each side
TRAINING    = 32     # training cells each side
MIN_PERSIST = 2      # consecutive frames required

detection_map = apply_cfar_full(
    psd, mu,
    guard=GUARD, training=TRAINING,
    pfa=PFA, min_persistence=MIN_PERSIST
)

print(f'Detection density: {detection_map.mean()*100:.2f}%')

## Step 4 – Event Grouping & Feature Extraction

In [ ]:
events = group_events(detection_map, psd, frequencies, timestamps, min_area=2)
events = classify_events(events)

df = pd.DataFrame(events)
print(f'Total events: {len(df)}')
print('\nTag distribution:')
print(df['tags'].str.split(',', expand=True).stack().value_counts())

df.head(10)

## Step 5 – Top 10 Events

In [ ]:
cols = ['event_id','f_centre_mhz','bandwidth_khz','peak_power_db',
        'duration_s','spectral_flatness','tags']

print('── Top 10 by Peak Power ──')
display(df.nlargest(10, 'peak_power_db')[cols])

print('\n── Top 10 by Bandwidth ──')
display(df.nlargest(10, 'bandwidth_hz')[cols])

## Step 6 – Occupancy Map

In [ ]:
occupancy = compute_occupancy(psd, mu, sigma, threshold_sigma=3.0)

import plotly.graph_objects as go
fig = go.Figure(go.Bar(
    x=frequencies/1e6, y=occupancy*100,
    name='Occupancy', marker_color='#44ccff'
))
fig.update_layout(
    title='Spectrum Occupancy by Frequency Bin',
    xaxis_title='Frequency (MHz)', yaxis_title='Occupancy (%)',
    template='plotly_dark', height=350
)
fig.show()

## Step 7 – Interactive Waterfall

In [ ]:
fig = plot_waterfall(
    psd, frequencies, timestamps,
    events=events,
    detection_map=detection_map,
    title=f'Spectrum Waterfall – {SENSOR_ID}'
)
fig.show()

## Step 8 – Feature Space Plot

In [ ]:
fig = plot_event_features(events)
fig.show()

## Step 9 – Save All Outputs

In [ ]:
import os
os.makedirs('../outputs', exist_ok=True)

save_events_csv(events, '../outputs/events.csv')
save_sensor_summary(events, psd, frequencies, timestamps, occupancy,
                    sensor_id=SENSOR_ID,
                    output_path='../outputs/sensor_summary.json')

plot_waterfall(psd, frequencies, timestamps, events=events,
               detection_map=detection_map,
               output_path='../outputs/band_waterfall.html')
plot_noise_floor(psd, mu, frequencies,
                 output_path='../outputs/noise_floor.html')
plot_event_features(events,
                    output_path='../outputs/event_features.html')

print('All outputs written to ../outputs/')